# Ejemplos Jax

## Imports

In [11]:
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap, random

## Ejemplo sencillo álgebra

In [19]:


# 1. Definición de la función
def f(x):
    return x**2 + 2*x + 1

# 2. Transformación 'grad' para obtener la derivada: f'(x) = 2x + 2
grad_f = jax.grad(f)

# 3. Transformación 'jit' para compilar la función
fast_f = jax.jit(f)

x_val = 3.0
print(f"Resultado de f(3): {fast_f(x_val)}")
print(f"Gradiente en x=3: {grad_f(x_val)}")

Resultado de f(3): 16.0
Gradiente en x=3: 8.0


## Ejemplo

In [20]:
# 1. Configuración de datos sintéticos
key = random.PRNGKey(42)
x_key, noise_key, params_key = random.split(key, 3)

# Generamos 100 puntos: y = 3x + 1 + ruido
x = random.normal(x_key, (100,))
noise = random.normal(noise_key, (100,)) * 0.1
y = 3 * x + 1 + noise

In [21]:
# 2. Inicialización de parámetros (W y b)
params = {
    "w": random.normal(params_key),
    "b": jnp.zeros(1)
}

In [22]:
# 3. Definición del modelo y función de pérdida
def predict(params, x):
    return params["w"] * x + params["b"]

def loss_fn(params, x, y):
    preds = predict(params, x)
    return jnp.mean((preds - y) ** 2)

In [23]:
# 4. Función de actualización acelerada con JIT
# Calculamos el gradiente de la pérdida respecto a los parámetros
@jit
def update(params, x, y, lr=0.1):
    grads = grad(loss_fn)(params, x, y)
    new_params = {
        "w": params["w"] - lr * grads["w"],
        "b": params["b"] - lr * grads["b"]
    }
    return new_params


In [24]:
# 5. Bucle de entrenamiento
print("Entrenando modelo:")
for epoch in range(50):
    params = update(params, x, y)
    if epoch % 10 == 0:
        current_loss = loss_fn(params, x, y)
        print(f"Época {epoch}: Pérdida = {current_loss:.6f}")

Entrenando modelo:
Época 0: Pérdida = 4.895490
Época 10: Pérdida = 0.069015
Época 20: Pérdida = 0.008671
Época 30: Pérdida = 0.007204
Época 40: Pérdida = 0.007157


In [25]:
# Resultados
print("-" * 30)
print(f"Valores finales -> W: {params['w']:.4f} (Real: 3.0), b: {params['b'][0]:.4f} (Real: 1.0)")

------------------------------
Valores finales -> W: 3.0054 (Real: 3.0), b: 0.9877 (Real: 1.0)
